In [1]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('datasets/bayes/mushrooms.csv')

In [6]:
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [8]:
df.shape

(8124, 23)

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [10]:
# Label Encoder converts the data into numeric form
le = LabelEncoder()

In [12]:
# Following function will be applied to each column
ds = df.apply(func = le.fit_transform)

In [13]:
ds.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [14]:
data = ds.values
X = data[:, 1:]
y = data[:, 0]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
# Naive Bayes algorithm is based on Bayes probability theorem
class CustomNB:
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    # Prior probability is probability of element belonging to particular class
    def prior_prob(self, label):
        # Total stores the total number of elements in training data
        total = self.y_train.shape[0]
        # Class examples store number of elements belonging to class with given label
        class_examples = np.sum(self.y_train == label)
        return class_examples/float(total)
    
    # Conditional probabilty is probabilty of element belonging to particular class have some feature
    def conditional_prob(self, feature_col, feature_val, label):
        # X filtered stores all the elements belonging to given class in training data
        X_filtered = self.X_train[self.y_train==label]
        numerator = np.sum(X_filtered[:, feature_col] == feature_val)
        denominator = len(X_filtered)
        # Probability is equal to number of elements of the given class having feature value equal to the given feature value for feature columns divided by total number of elements in the class
        return numerator / denominator
    
    def predict_point(self, X_test):
        classes = np.unique(self.y_train) 
        n_features = self.X_train.shape[1]
        post_pro = []
        # Stores the probablity for element belonging each and every class and class with max probability is the answer
        for label in classes:
            likehood = 1.0
            # To find the probaility of element belonging to particular class with the set of features, conditional probability for each feature is multiplied assuming features are independent of each other
            for feature in range(n_features):
                cond = self.conditional_prob(feature, X_test[feature], label)
                likehood *= cond
            prior = self.prior_prob(label)
            # Find post_prob using Bayes Probability Theorem (P(A/B) = (P(B/A) * P(A))/P(B)), here we ignore P(B) in denominator as it will be constant in all the terms
            post = prior * likehood
            post_pro.append(post)
        
       # return the index of the largest probability value in array which gives the class label with max probability
        return np.argmax(post_pro) 
    
    def predict(self, X_test):
        result = []
        for point in X_test:
            result.append(self.predict_point(point))
        return np.array(result)
    
    def score(self, X_test, y_test):
        return (self.predict(X_test) == y_test).mean()*100

In [24]:
model = CustomNB()

In [25]:
model.fit(X_train, y_train)

In [26]:
model.predict(X_test[:10])

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0], dtype=int64)

In [27]:
y_test[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [28]:
model.score(X_test, y_test)

99.73890339425587